#Demonstration of atomman.lammps code design

__The atomman.lammps module has a number of tools for interactions with the LAMMPS molecular dynamics program. The principle actions of atomman.lammps are:__

1. Generating the LAMMPS input script lines associated with a given interatomic potential.

2. Generating the LAMMPS input script lines associated with creating a new system. 

3. Exchanging information to/from LAMMPS.

__These actions allow for the integration of LAMMPS simulations into a Python framework to facilitate rapid development of new simulations and easy adaption to different potentials or material systems.__

##0. Initial Setup

###0.1 Python package imports

In [1]:
#Standard Python Libraries
import subprocess
import os

import numpy as np

#Custom package
import atomman

###0.2 LAMMPS Executable

__Running LAMMPS simulations requires having a LAMMPS executable accessible.  This needs to be specified for your system.__

In [2]:
#Specify which LAMMPS executible to use
lammps_exe = 'C:/Users/lmh1/Documents/lmp_serial.exe'

#Test that lammps_exe is a file
if not os.path.isfile(lammps_exe):
    raise ValueError('Could not find LAMMPS executable ' + lammps_exe)

##1.  Representing a LAMMPS potential

__The LAMMPS molecular dynamics program is capable of performing simulations and calculations using a wide variety of interatomic potentials.  To facilitate this, the LAMMPS input scripts allow for the specification of numerous parameters that define or influence the interatomic interaction models.  Across potentials, the values, types and numbers of necessary parameters may drastically vary.  However, an implementation of a potential into LAMMPS is associated with a particular set of parameters that fully defines that atomistic model.__

__The parameter set associated with running LAMMPS with a particular potential implementation can be collected together into a data model.  For AtomMan, a json/XML based data model is used that is capable of fully representing the broad spectrum of interatomic potential parameters in a format that is easily accessible both for human and machine readability. A class atomman.lammps.Potential reads in data model files and handles all conversions, etc. using methods.  This Potential class, along with it being decoupled from System, allows for different interatomic potentials to be easily assigned and exchanged.__

###1.1 Example data models

__This section contains example potential instance data models.  More information on their structure and design can be found in the AtomMan LAMMPS Potential Notebook.__

__THESE ARE NOT REAL POTENTIALS! ONLY FOR DEMONSTRATION PURPOSES!__

###1.1.1 Simple style potential, i.e. Lennard-Jones.

In [3]:
#Creates an interatomic potential data model consistent with an lj/cut potential
f = open('lj-example.json', 'w')
f.write("""{
    "interatomicPotentialImplementationLAMMPS": {
        "potentialID": {
            "descriptionIdentifier": "lj-example"
        },
        "units": "lj",
        "atom_style": "atomic",
        "atom": [
            {
                "element": "He"
            },
            {
                "element": "Ar"
            }
        ],
        "pair_style": {
            "type": "lj/cut",
            "term": {
                "option": 10.0
            }
        },
        "pair_coeff": [
            {
                "interaction": {
                    "element": [
                        "He",
                        "He"
                    ]
                },
                "term": {
                    "option": "1.0 1.0"
                }
            },
            {
                "interaction": {
                    "element": [
                        "Ar",
                        "Ar"
                    ]
                },
                "term": {
                    "option": "2.0 2.0"
                }
            },
            {
                "interaction": {
                    "element": [
                        "He",
                        "Ar"
                    ]
                },
                "term": {
                    "option": "1.0 2.0"
                }
            }            
        ]
    }
}""")
f.close()      

###1.1.2 Many-body potential with an associated potential parameter file, i.e. eam/alloy.

In [4]:
#Creates an interatomic potential data model consistent with an eam/alloy potential
f = open('eam-alloy-example.json', 'w')
f.write("""{
    "interatomicPotentialImplementationLAMMPS": {
        "potentialID": {
            "descriptionIdentifier": "eam-alloy-example"
        },
        "units": "metal",
        "atom_style": "atomic",
        "atom": [
            {
                "element": "Ni",
                "mass": 58.6934
            },
            {
                "element": "Al",
                "mass": 26.981539
            },
            {
                "element": "Co",
                "mass": 58.9332
            }
        ],
        "pair_style": {
            "type": "eam/alloy"
        },
        "pair_coeff": {
            "term": [
                {
                    "file": "file.eam.alloy"
                },
                {
                    "symbols": "True"
                }
            ]
        }
    }
}""")
f.close()      

###1.2 Potential class usage

__This section contains code demonstrating the usage of the atomman.lammps.Potential class to generate script information from a potential instance data model file.__

__A potential instance data model file can be easily read in creating a Potential object instance.__

In [5]:
#pot = atomman.lammps.Potential('lj-example.json')
#or
pot = atomman.lammps.Potential('eam-alloy-example.json')

#The str() of the class returns the potential's name.
print 'Potential is:', pot

Potential is: eam-alloy-example


__The units and atom_style can be easily retrieved using appropriately named methods:__

In [6]:
print 'units', pot.units()
print 'atom_style', pot.atom_style()

units metal
atom_style atomic


__The elements(), symbols() and masses() methods return lists of the associated values.  elements() and masses() also can take arguments to return a specific value or list of values associated with a particular element symbol or list of symbols.__

In [7]:
symbols = pot.symbols()

print 'Default method calls:'
print 'symbols =',  symbols
print 'elements =', pot.elements()
print 'masses =',   pot.masses()
print

print 'Values using a single specified element symbol:'
for i in xrange(len(symbols)):
    print 'Mass of', pot.elements(symbols[i]), 'is', pot.masses(symbols[i])
print

#Create arbitrary list of symbols for demonstration purposes
symbol_list = [symbols[0], symbols[0], symbols[1], symbols[1], symbols[0], symbols[1]]
print 'Values using the symbol list', symbol_list
print 'elements =', pot.elements(symbol_list)
print 'masses =',   pot.masses(symbol_list)

Default method calls:
symbols = [u'Ni', u'Al', u'Co']
elements = [u'Ni', u'Al', u'Co']
masses = [58.6934, 26.981539, 58.9332]

Values using a single specified element symbol:
Mass of Ni is 58.6934
Mass of Al is 26.981539
Mass of Co is 58.9332

Values using the symbol list [u'Ni', u'Ni', u'Al', u'Al', u'Ni', u'Al']
elements = [u'Ni', u'Ni', u'Al', u'Al', u'Ni', u'Al']
masses = [58.6934, 58.6934, 26.981539, 26.981539, 58.6934, 26.981539]


__Finally, a method pair_info() generates the mass, pair_style and pair_coeff lines__ 

In [8]:
print 'Default pair_info:'
print pot.pair_info()
print
print 'pair_info with one element symbol specified:'
print pot.pair_info(symbols[0])
print

print 'pair_info using the symbol list', symbol_list
print pot.pair_info(symbol_list)

Default pair_info:
mass 1 58.693400
mass 2 26.981539
mass 3 58.933200

pair_style eam/alloy
pair_coeff * * file.eam.alloy Ni Al Co


pair_info with one element symbol specified:
mass 1 58.693400

pair_style eam/alloy
pair_coeff * * file.eam.alloy Ni


pair_info using the symbol list [u'Ni', u'Ni', u'Al', u'Al', u'Ni', u'Al']
mass 1 58.693400
mass 2 58.693400
mass 3 26.981539
mass 4 26.981539
mass 5 58.693400
mass 6 26.981539

pair_style eam/alloy
pair_coeff * * file.eam.alloy Ni Ni Al Al Ni Al



##2. Generating new systems using LAMMPS.

__AtomMan also contains a function atomman.lammps.sys_gen() that generates the LAMMPS input script lines associated with creating a new system.  The available arguments are:__

- units = the LAMMPS units type.

- atom_style = the LAMMPS atom_style type.

- pbc = list of three boolean values indicating which directions are periodic.

- ucell_box = an atomman.Box representing the unit cell size and dimensions.

- ucell_atoms = a list of atomman.Atom instances in the unit cell.

- axes = the cubic crystallographic axes to rotate the system by. Note that this will rotate any system as if the axes were a transformation matrix, but will only correspond to crystallographic axes for cubic systems.

- shift = a shift of atomic positions.

- size = multipliers used in generating the system.

__Note: This funcion simply creates the script telling LAMMPS to generate a block of atoms.  As such, it has the same limitations as LAMMPS does when generating atoms (most notably some issues at periodic boundaries).__

__Note 2: The spacing value differs from the default value calculated by LAMMPS.  The values used by AtomMan "should" give periodic systems for rotated cubic systems, and non-rotated triclinic systems.__  

In [9]:
#Call to sys_gen with all default argument values listed.
system_info = atomman.lammps.sys_gen(units = 'metal', 
                                     atom_style = 'atomic', 
                                     pbc = (True, True, True), 
                                     ucell_box = atomman.Box(),
                                     ucell_atoms = [atomman.Atom(1, np.array([0.0, 0.0, 0.0])),
                                                    atomman.Atom(1, np.array([0.5, 0.5, 0.0])),
                                                    atomman.Atom(1, np.array([0.0, 0.5, 0.5])),
                                                    atomman.Atom(1, np.array([0.5, 0.0, 0.5]))],
                                     axes = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
                                     shift = np.array([0.1, 0.1, 0.1]),
                                     size = np.array([[-3,3], [-3,3], [-3,3]], dtype=np.int))

print system_info

#Atomic system info generated by AtomMan package

units metal
atom_style atomic
boundary p p p 

lattice custom 1.0 &
        a1 1.000000000000 0.000000000000 0.000000000000 &
        a2 0.000000000000 1.000000000000 0.000000000000 &
        a3 0.000000000000 0.000000000000 1.000000000000 &
        origin 0.100000 0.100000 0.100000 &
        spacing 1.000000000000 1.000000000000 1.000000000000 &
        orient x 1 0 0 &
        orient y 0 1 0 &
        orient z 0 0 1 &
        basis 0.000000 0.000000 0.000000 &
        basis 0.500000 0.500000 0.000000 &
        basis 0.000000 0.500000 0.500000 &
        basis 0.500000 0.000000 0.500000

region box block -3 3 -3 3 -3 3
create_box 1 box
create_atoms 1 box


##3. Interacting with LAMMPS atomic data

__AtomMan handles the exchange of information with LAMMPS by reading atomic data files, dump files and log files, and writing atomic data files, dump files and input scripts.__  

###3.1 Writing System information to an atomic data file.

__Atomic data files can be created using atomman.lammps.write_data().  For arguments, the function takes a file name, an atomman.System, the units type, and the atom_style type.__

In [10]:
#Create a generic system
system = atomman.System(box=atomman.Box(a=3.2, b=3.2, c=3.2),
                        atoms=[atomman.Atom(1, [0.0, 0.0, 0.0]),
                               atomman.Atom(1, [0.5, 0.5, 0.5])], 
                        pbc=(True, True, True),
                        scale=True)

#Write to an atomic style data file
system_info = atomman.lammps.write_data('system.data', system, units='metal', atom_style='atomic')

print "Contents of system.data are:"
with open('system.data') as f:
    print f.read()

Contents of system.data are:

2 atoms
1 atom types
0.000000 3.200000 xlo xhi
0.000000 3.200000 ylo yhi
0.000000 3.200000 zlo zhi

Atoms

1 1 0.0000000000000e+00 0.0000000000000e+00 0.0000000000000e+00
2 1 1.6000000000000e+00 1.6000000000000e+00 1.6000000000000e+00



__write_data() also returns a string containing the input script lines associated with having LAMMPS read the data file.__

In [11]:
print system_info

#Script and data file prepared by AtomMan package

units metal
atom_style atomic
boundary p p p 
read_data system.data


###3.2 Reading System information from an atomic data file.

__If you ned to read the atomic information back in from an atomic data file you can use atomman.lammps.read_data().  For arguments, the function takes a file name, the periodic boundary conditions (pbc), and the atom_style type.  atom_style is necessary to properly interpret the data, and pbc is needed as the data files do not include this information.__

In [12]:
system_check = atomman.lammps.read_data('system.data', pbc=(True, True, True), atom_style='atomic')

print 'a =    ', system_check.box('a')
print 'b =    ', system_check.box('b')
print 'c =    ', system_check.box('c')
print 'alpha =', system_check.box('alpha')
print 'beta = ', system_check.box('beta')
print 'gamma =', system_check.box('gamma')
print
print 'pbc =', system_check.pbc()
print 
for i in xrange(system_check.natoms()):
    print system_check.atoms(i, 'atype'), system_check.atoms(i, 'pos') 

a =     3.2
b =     3.2
c =     3.2
alpha = 90.0
beta =  90.0
gamma = 90.0

pbc = (True, True, True)

1 [ 0.  0.  0.]
1 [ 1.6  1.6  1.6]


###3.3 Writing System information to a dump file.

__Working with the atomic data files is useful for setting up simulations, but post-analysis is better apt to interacting with dump style files.  A System's atomic information can be written to a dump file using atomman.lammps.write_dump().__ 

In [13]:
#assign additional per-atom properties to the system
for i in xrange(2):
    system.atoms(i, 'scalar', 0.0)
    system.atoms(i, 'vector', [1.0, 2.0, 3.0])
    system.atoms(i, 'matrix', [[11, 12], [21, 22]])
    
atomman.lammps.write_dump('system.dump', system)

print "Contents of system.dump are:"
with open('system.dump') as f:
    print f.read()

Contents of system.dump are:
ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
2
ITEM: BOX BOUNDS pp pp pp
0.000000 3.200000
0.000000 3.200000
0.000000 3.200000
ITEM: ATOMS id type x y z scalar vector[0] vector[1] vector[2] matrix[0][0] matrix[0][1] matrix[1][0] matrix[1][1]
1 1 0.0000000000000e+00 0.0000000000000e+00 0.0000000000000e+00 0.0000000000000e+00 1.0000000000000e+00 2.0000000000000e+00 3.0000000000000e+00 11 12 21 22
2 1 1.6000000000000e+00 1.6000000000000e+00 1.6000000000000e+00 0.0000000000000e+00 1.0000000000000e+00 2.0000000000000e+00 3.0000000000000e+00 11 12 21 22



__write_dump() has two optional arguments.  xf allows for the format of floating point numbers to be changed by specifying a c-style format string.  scale=True converts the atomic positions to be relative to the system box.__

In [14]:
atomman.lammps.write_dump('system.dump', system, xf='%.3f', scale=True)

print "Contents of system.dump are:"
with open('system.dump') as f:
    print f.read()

Contents of system.dump are:
ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
2
ITEM: BOX BOUNDS pp pp pp
0.000000 3.200000
0.000000 3.200000
0.000000 3.200000
ITEM: ATOMS id type xs ys zs scalar vector[0] vector[1] vector[2] matrix[0][0] matrix[0][1] matrix[1][0] matrix[1][1]
1 1 0.000 0.000 0.000 0.000 1.000 2.000 3.000 11 12 21 22
2 1 0.500 0.500 0.500 0.000 1.000 2.000 3.000 11 12 21 22



###3.4 Reading System information from a dump file.

__Finally, a System can be constructed by reading from a dump file using atomman.lammps.read_dump().__

In [15]:
system_check = atomman.lammps.read_dump('system.dump')

print 'a =    ', system_check.box('a')
print 'b =    ', system_check.box('b')
print 'c =    ', system_check.box('c')
print 'alpha =', system_check.box('alpha')
print 'beta = ', system_check.box('beta')
print 'gamma =', system_check.box('gamma')
print
print 'pbc =', system_check.pbc()
print 
for i in xrange(system_check.natoms()):
    print 'Atom #:', i+1
    print 'atype = ', system_check.atoms(i, 'atype')
    print 'pos =   ', system_check.atoms(i, 'pos')
    print 'scalar =', system_check.atoms(i, 'scalar')
    print 'vector =', system_check.atoms(i, 'vector')
    print 'matrix ='
    print system_check.atoms(i, 'matrix') 
    print

a =     3.2
b =     3.2
c =     3.2
alpha = 90.0
beta =  90.0
gamma = 90.0

pbc = (True, True, True)

Atom #: 1
atype =  1
pos =    [ 0.  0.  0.]
scalar = 0.0
vector = [ 1.  2.  3.]
matrix =
[[11 12]
 [21 22]]

Atom #: 2
atype =  1
pos =    [ 1.6  1.6  1.6]
scalar = 0.0
vector = [ 1.  2.  3.]
matrix =
[[11 12]
 [21 22]]



##4. Running LAMMPS

###4.1 Generating LAMMPS input scripts

__The previously mentioned commands make generating and reusing LAMMPS input scripts easy.  All input lines associated with a given system can be obtained from either sys_gen() or write_data(), and all input lines associated with an interatomic potential can be obtained from Potential.pair_info()__

In [17]:
#Creates an interatomic potential data model with no resulting interaction
f = open('none.json', 'w')
f.write("""{
    "interatomicPotentialImplementationLAMMPS": {
        "potentialID": {
            "descriptionIdentifier": "none"
        },
        "units": "metal",
        "atom_style": "atomic",
        "atom": {
            "element": "X",
            "mass": 1.0
        },
        "pair_style": {
            "type": "morse 0.001"
        },
        "pair_coeff": {
            "term": {
                "option": "0.0 0.0 0.0"
            }
        }
    }
}""")
f.close() 

#Create a Potential associated with the none.json data model
none_pot = atomman.lammps.Potential('none.json')
pair_info = none_pot.pair_info()

#Use the default sys_gen() to build an atomic system
system_info = atomman.lammps.sys_gen(units=none_pot.units(), atom_style=none_pot.atom_style())

#Show system_info and pair_info
print system_info
print pair_info

#Atomic system info generated by AtomMan package

units metal
atom_style atomic
boundary p p p 

lattice custom 1.0 &
        a1 1.000000000000 0.000000000000 0.000000000000 &
        a2 0.000000000000 1.000000000000 0.000000000000 &
        a3 0.000000000000 0.000000000000 1.000000000000 &
        origin 0.100000 0.100000 0.100000 &
        spacing 1.000000000000 1.000000000000 1.000000000000 &
        orient x 1 0 0 &
        orient y 0 1 0 &
        orient z 0 0 1 &
        basis 0.000000 0.000000 0.000000 &
        basis 0.500000 0.500000 0.000000 &
        basis 0.000000 0.500000 0.500000 &
        basis 0.500000 0.000000 0.500000

region box block -3 3 -3 3 -3 3
create_box 1 box
create_atoms 1 box
mass 1 1.000000

pair_style morse 0.001
pair_coeff * * 0.0 0.0 0.0



__From this, LAMMPS input scripts can be developed that are independent of the potential and/or the system.__

In [23]:
def run10_script(system_info, pair_info):
    #Performs a run0 energy calculation
    nl = '\n'
    script = nl.join([system_info,
                      '',
                      pair_info,
                      '',
                      'compute peatom all pe/atom',
                      ''
                      'thermo 1',
                      'thermo_style custom step pe',
                      'thermo_modify format float %.13e',
                      '',
                      'dump dumpit all custom 100 atom.0 id type x y z c_peatom',
                      'dump_modify dumpit format "%i %i %.13e %.13e %.13e %.13e"',
                      '',
                      'run 10'])
    return script

input_script = run10_script(system_info, pair_info)
print input_script

#Atomic system info generated by AtomMan package

units metal
atom_style atomic
boundary p p p 

lattice custom 1.0 &
        a1 1.000000000000 0.000000000000 0.000000000000 &
        a2 0.000000000000 1.000000000000 0.000000000000 &
        a3 0.000000000000 0.000000000000 1.000000000000 &
        origin 0.100000 0.100000 0.100000 &
        spacing 1.000000000000 1.000000000000 1.000000000000 &
        orient x 1 0 0 &
        orient y 0 1 0 &
        orient z 0 0 1 &
        basis 0.000000 0.000000 0.000000 &
        basis 0.500000 0.500000 0.000000 &
        basis 0.000000 0.500000 0.500000 &
        basis 0.500000 0.000000 0.500000

region box block -3 3 -3 3 -3 3
create_box 1 box
create_atoms 1 box

mass 1 1.000000

pair_style morse 0.001
pair_coeff * * 0.0 0.0 0.0


compute peatom all pe/atom
thermo 1
thermo_style custom step pe
thermo_modify format float %.13e

dump dumpit all custom 100 atom.0 id type x y z c_peatom
dump_modify dumpit format "%i %i %.13e %.13e %.13e %.13e"

run

###4.2 Calling LAMMPS and processing the results

__The standard Python library subprocess allows for LAMMPS to be called directly from a Python script or an IPython Notebook.  In particular, subprocess.check_output() will run the argument and return the standard screen output.  This output (or a log.lammps file) can be parsed for the thermo information using atomman.lammps.log_extract().__ 

In [24]:
#Write script to a file
with open('test.in', 'w') as f:
    f.write(input_script)
    
#Run LAMMPS
output = subprocess.check_output(lammps_exe + ' -in ' + 'test.in')
data = atomman.lammps.log_extract(output)

__The extracted thermo data is stored in a 2D list. The first row is the names of the thermo properties.  All other rows are the values assigned to those properties.__

__NOTE: If your LAMMPS input script runs multiple simulations the thermo values are all collected into one 2D list.  Inherent to this is the assumption that the thermo style is the same for all simulations.__

In [26]:
for row in data:
    print row

['Step', 'PotEng']
['0', '0.0000000000000e+000']
['1', '0.0000000000000e+000']
['2', '0.0000000000000e+000']
['3', '0.0000000000000e+000']
['4', '0.0000000000000e+000']
['5', '0.0000000000000e+000']
['6', '0.0000000000000e+000']
['7', '0.0000000000000e+000']
['8', '0.0000000000000e+000']
['9', '0.0000000000000e+000']
['10', '0.0000000000000e+000']


__Demo of extracting data from log.lammps instead of from standard output.__

In [27]:
with open('log.lammps') as f:
    data = atomman.lammps.log_extract(f.read())

for row in data:
    print row    

['Step', 'PotEng']
['0', '0.0000000000000e+000']
['1', '0.0000000000000e+000']
['2', '0.0000000000000e+000']
['3', '0.0000000000000e+000']
['4', '0.0000000000000e+000']
['5', '0.0000000000000e+000']
['6', '0.0000000000000e+000']
['7', '0.0000000000000e+000']
['8', '0.0000000000000e+000']
['9', '0.0000000000000e+000']
['10', '0.0000000000000e+000']
